In [2]:
#import library
import collections
import random
import os
import time
import json
from PIL import Image
import io
import urllib
import uuid
from concurrent.futures import ThreadPoolExecutor
from functools import partial

import numpy as np
from tqdm import tqdm
from datasets import load_dataset
from datasets.utils.file_utils import get_datasets_user_agent
import matplotlib.pyplot as plt
import tensorflow as tf

#import other packages for dataset store
import pandas as pd
from datasets import Dataset, DatasetDict, load_from_disk

In [9]:
#split test data based on hau, tha, kir
total_test = pd.read_csv("test.csv")
test_hau = total_test[total_test['ISO639-3']=='hau'] #64
test_tha = total_test[total_test['ISO639-3']=='tha'] #69
test_kir = total_test[total_test['ISO639-3']=='kir'] #67

In [17]:
#save seperately
import csv

header = ["'Id', 'ImageURL', 'ISO639-3'"]
test_hau.to_csv("images_test_hau.csv",index=False)
test_tha.to_csv("images_test_tha.csv",index=False)
test_kir.to_csv("images_test_kir.csv",index=False)

In [18]:
test_dataset_hau = load_dataset('csv', data_files='images_test_hau.csv')
test_dataset_tha = load_dataset('csv', data_files='images_test_tha.csv')
test_dataset_kir = load_dataset('csv', data_files='images_test_kir.csv')

Using custom data configuration default-27f7657afc698e1f


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:/Users/Zhiwen Yan/.cache/huggingface/datasets/csv/default-27f7657afc698e1f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

...

Using custom data configuration default-009d5bab45ad0376


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:/Users/Zhiwen Yan/.cache/huggingface/datasets/csv/default-009d5bab45ad0376/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
#actual download image from url
USER_AGENT = get_datasets_user_agent()

def fetch_single_image(image_url, timeout=None, retries=0):
    request = urllib.request.Request(
        image_url,
        data=None,
        headers={"user-agent": USER_AGENT},
    )
    with urllib.request.urlopen(request, timeout=timeout) as req:
        if 'png' in image_url:
          png = Image.open(io.BytesIO(req.read())).convert('RGBA')
          png.load() # required for png.split()
          background = Image.new("RGB", png.size, (255, 255, 255))
          background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
          image_id = str(uuid.uuid4()) # confused about image_id here?
          image_path = "images_kir_test/" + image_id + ".jpg"
          background.save(image_path, 'JPEG', quality=80)
        else:
          image = Image.open(io.BytesIO(req.read()))
          image_id = str(uuid.uuid4())
          image_path = "images_kir_test/" + image_id + ".jpg"
          image.save(image_path)
    return image_path

def fetch_images(batch, num_threads, timeout=None, retries=3):
    fetch_single_image_with_args = partial(fetch_single_image, timeout=timeout, retries=retries)
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        batch["image_path"] = list(executor.map(fetch_single_image_with_args, batch["ImageURL"]))
    return batch

num_threads = 20
test_dataset_kir = test_dataset_kir.map(fetch_images, batched=True, batch_size=100, fn_kwargs={"num_threads": num_threads})

  0%|          | 0/1 [00:00<?, ?ba/s]

In [29]:
#save dataset to local
test_dataset_kir.save_to_disk("dataset_kir_test")

### Reload data and extract image feature by InceptionV3

In [43]:
#reload dataset for hau tha kir
dataset = load_from_disk("dataset_kir_test")

In [44]:
dataset 
#note although it is train here, its actually test dataset

DatasetDict({
    train: Dataset({
        features: ['Id', 'ImageURL', 'ISO639-3', 'image_path'],
        num_rows: 67
    })
})

In [32]:
# Retrieve the InceptionV3 model
# Prepare images features with a pre-trained InceptionV3 model
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [34]:
# Cache image features
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.keras.layers.Resizing(299, 299)(img)
    img = tf.keras.applications.inception_v3.preprocess_input(img) #1 299 299 3
    return img, image_path

In [45]:
# Get unique train(actually test!) images
encode_train = sorted(set(dataset['train']['image_path']))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.AUTOTUNE).batch(16)

for img, path in tqdm(image_dataset):
  batch_features = image_features_extract_model(img) #16 8 8 2048
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3])) #1 64 2048

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())
    

100%|██████████| 5/5 [00:03<00:00,  1.38it/s]
